<a href="https://colab.research.google.com/github/Fuzz117/MLBLR/blob/master/session62.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import re
import string
from google.colab import drive
drive.mount("/content/gdrive")



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
filename = "/content/gdrive/My Drive/Colab Notebooks/eip3p2/session6.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
raw_text = re.sub('['+string.punctuation+']', '', raw_text)


raw_text=raw_text.split("\n\n")

raw_text = [x for x in raw_text if x is not ''] 



In [0]:
N = 100
new_text = []
for text in raw_text:
  text += ' ' * (N - len(text))
  new_text.append(text)
new_text = ' '.join(new_text)

In [0]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(new_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [24]:
# summarize the loaded data
n_chars = len(new_text)
n_vocab = len(chars)
print ("Total Characters: "), n_chars
print ("Total Vocab: "), n_vocab
print (chars)

Total Characters: 
Total Vocab: 
['\n', ' ', '0', '3', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [25]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = new_text[i:i + seq_length]
	seq_out = new_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: "), n_patterns

Total Patterns: 


(None, 151651)

In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))

model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 30)                7710      
Total params: 797,214
Trainable params: 797,214
Non-trainable params: 0
_________________________________________________________________


In [0]:
filepath="/content/gdrive/My Drive/Colab Notebooks/eip3p2/weights1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [0]:
# model = create_model()
# model.load_weights("/content/gdrive/My Drive/Colab Notebooks/eip3p2/weights1.hdf5")


In [33]:
# fit the model
model.fit(X, y, epochs=100, batch_size=1024, callbacks=callbacks_list)


Epoch 1/100
151651/151651 [==============================] - 121s 795us/step - loss: 2.5870 - acc: 0.2788

Epoch 00001: loss improved from 2.68608 to 2.58701, saving model to /content/gdrive/My Drive/Colab Notebooks/eip3p2/weights1.hdf5
Epoch 2/100
151651/151651 [==============================] - 121s 796us/step - loss: 2.5213 - acc: 0.2880

Epoch 00002: loss improved from 2.58701 to 2.52131, saving model to /content/gdrive/My Drive/Colab Notebooks/eip3p2/weights1.hdf5
Epoch 3/100
151651/151651 [==============================] - 121s 797us/step - loss: 2.3835 - acc: 0.3386

Epoch 00003: loss improved from 2.52131 to 2.38352, saving model to /content/gdrive/My Drive/Colab Notebooks/eip3p2/weights1.hdf5
Epoch 4/100
151651/151651 [==============================] - 121s 800us/step - loss: 2.2576 - acc: 0.3584

Epoch 00004: loss improved from 2.38352 to 2.25764, saving model to /content/gdrive/My Drive/Colab Notebooks/eip3p2/weights1.hdf5
Epoch 5/100
151651/151651 [=========================

In [34]:

model.save("/content/gdrive/My Drive/Colab Notebooks/eip3p2/weights1.h5",overwrite='true')
print("Saved model to disk")

Saved model to disk


In [35]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(result,end='')
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" ring
itself upright as it spoke it was exactly three inches high but im not used to it pleaded poor  "
alice in a great hurry and their saie the cock in a thry nole of liven she euece in the mouse and jost as she sabbit began alice gallening the sight there was no time to be said to ierself as she said the queen thi macbit were all this time when they have to she shought that it might be lurt said the caterpillar                                                                             it isnt directed the kittle shisg rame mooe surned to alice replied                                         th
Done.
